In [1]:
# pip install torch

In [2]:
# pip install tqdm


In [3]:
#tutorial followed: https://www.youtube.com/watch?v=UU1WVnMk4E8

In [4]:
# pip install pytorch==1.6.0 torchvision==0.7.0 cudatoolkit=12.3 -c pytorch


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [6]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [7]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)



inputs:
tensor([[18, 11,  0,  0, 44, 32, 29,  1],
        [61, 58,  1, 61, 54, 57,  1, 72],
        [ 1, 73, 68,  1, 72, 58, 58,  1],
        [74, 65, 58, 71,  1, 68, 59,  1]], device='cuda:0')
targets:
tensor([[11,  0,  0, 44, 32, 29,  1, 37],
        [58,  1, 61, 54, 57,  1, 72, 56],
        [73, 68,  1, 72, 58, 58,  1, 78],
        [65, 58, 71,  1, 68, 59,  1, 73]], device='cuda:0')


In [9]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



C&_JFg)jxT)m.wAM T:[_BZ2]6M6BHcWTsJ,RZ﻿QaRM*aZVBeO]IS8sP&8gHZIxVo"o6oOu'AjIk:[Okp&kg;oE-UW&2.Lx3fPphIajJ_,R;oDbqr.ng2I5dQ:SpwZQohF*s?d_J[L8mc*k8xrPqr-]G"c2)mKjdpw!T:b,C_J
Gmxs;bQ[3EyR
3VpuVrNWzaKLq2XdQ;'ACWn﻿RVz"RE:?SjJaIw:c:n(N-iMkP
k:8Wn:0jxsx,;oEFY;ozy_ca!K5!p(0*g(7SdD onYN(WnSf8"&7a&,J.M6MOtr﻿_;'!7;Ue2:[]Jr﻿pnlh:n)r﻿0F96f.XU7.:N'"RQa!_gB﻿BbA*zGNV4h:n[WdEUtEo8mhd)?)??'D*M]Z7C4h:﻿r:T; x)RYUhXvt﻿wVZ*g:MOf6FTV4Ff--﻿VbbT43d4Bv﻿9JHxBprmIJB(re-x5;MhwgeORlcp:kG7DUxg;;o1tH8'2dc:_e;UF&8gjs1&FHsf'Sju&7


In [11]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.081, val loss: 5.106
step: 250, train loss: 5.053, val loss: 5.039
step: 500, train loss: 4.969, val loss: 4.994
step: 750, train loss: 4.925, val loss: 4.932
4.7083234786987305


***need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them***

1. **Mean Squared Error (MSE)**: MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. **Gradient Descent (GD):**  is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. **Momentum**: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. **RMSprop**: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. **Adam**: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. **AdamW**: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim

In [12]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


YuX.wcs
G*v7V-5Z1sV﻿]bbb_.waGz2.AA2KfuDk!Hs9.wUfYr1T)b3]EF3c sW6
I57Df5GmSj*(WcPYaIalbXbpVHTm9.wa"49G]_z"&'A:S40MwM0jKT8A,CIq-*1!p,R;:Dke(m
q_zD'xwQ)TprrMAErng_gAGV?XR.7exZm:]0jc kge?EPlcbKaIP2&,JDYTxRy:!9b5vt'44hSLj)'(wi1Y,﻿wdw:zZ3 yEa[]F'x1tGP!JIM8p-*V0jJ﻿VI"lp3uD5v_r*FOTUJu&8p[h1!gMi7DJAvs')bw_﻿sUK?GN;o)(ySX7D H4q80OsWP'tN;ANA3U:hzIXdngIX A9a_la:(l7DwV ')p(.5!!)? [I4bPo"wAv(iKfrPbW?jJr&on6 ﻿
Y,JD!IjT0DRNwACSuy6-﻿ep,hfuWwX? kXGW0A,l849zJq,m*Gq[_tZBD!HBqd;8pY0i),3myZ'4DXqk7'kgFdI5bhz3xtOdQ,pw6F
